In [21]:
import torch
from torchvision import datasets
import torchvision.transforms as transform
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.optim as optim
import torch.nn as nn
import numpy as np
from ultralytics import YOLO
from torchvision.datasets import ImageFolder

In [22]:
class CustomAugmentedDataset(ImageFolder):

    # constructor
    def __init__(self, root, transforms_dict, default_transforms=None):
        super().__init__(root)
        self.transform_dict = transforms_dict
        self.default_transform = default_transforms

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)

        class_name = self.classes[target]
        transform = self.transform_dict.get(class_name, self.default_transform)

        if transform:
            sample = transform(sample)
        return sample, target

In [23]:
default_transform = transform.Compose([
    transform.Resize(256),
    transform.CenterCrop(224),
    transform.ToTensor(),
    transform.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

strong_transform = transform.Compose([
    transform.Resize(256),
    transform.RandomResizedCrop(224),
    transform.RandomHorizontalFlip(),
    transform.RandomRotation(30),
    transform.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transform.ToTensor(),
    transform.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

small_classes = ['red-spot']
transform_dict = {cls: strong_transform for cls in small_classes}

# load dataset beserta transform-nya
train_dataset = CustomAugmentedDataset(root='/home/oz31/code/personal/python/tea/dataset/Train/', transforms_dict=transform_dict, default_transforms=default_transform)
val_dataset = CustomAugmentedDataset(root='/home/oz31/code/personal/python/tea/dataset/Valid/', transforms_dict=transform_dict, default_transforms=default_transform)

valid_classes = ['algal_spot', 'brown-blight', 'gray-blight', 'healthy', 'helopeltis', 
                 'leaf-rust', 'red-rust', 'red-spider-infested', 'red-spot', 'white-spot']
train_class_to_idx = {cls: idx for idx, cls in enumerate(valid_classes) if cls in train_dataset.classes}
train_samples = [(path, train_class_to_idx[train_dataset.classes[label]]) 
                 for path, label in train_dataset.samples 
                 if train_dataset.classes[label] in valid_classes]
val_samples = [(path, train_class_to_idx[val_dataset.classes[label]]) 
               for path, label in val_dataset.samples 
               if val_dataset.classes[label] in valid_classes]

if not train_samples or not val_samples:
    raise ValueError("No samples match the 10 classes. Check dataset subfolders.")

train_dataset.samples = train_samples
train_dataset.classes = valid_classes
train_dataset.class_to_idx = train_class_to_idx
val_dataset.samples = val_samples
val_dataset.classes = valid_classes
val_dataset.class_to_idx = train_class_to_idx

class_counts = np.bincount([label for _, label in train_dataset.samples])
class_weight = 1. / torch.tensor(class_counts, dtype=torch.float)

sample_weights = [class_weight[label] for _, label in train_dataset.samples]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_dataloader = DataLoader(train_dataset, batch_size=8, sampler=sampler, num_workers=6, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, num_workers=4, shuffle=False)

# Log sample counts
from collections import Counter

train_counts = Counter(train_dataset.classes[label] for _, label in train_samples)
val_counts = Counter(val_dataset.classes[label] for _, label in val_samples)
print("Train sample counts per class:")
for cls in valid_classes:
    print(f"{cls}: {train_counts.get(cls, 0)}")
print("Validation sample counts per class:")
for cls in valid_classes:
    print(f"{cls}: {val_counts.get(cls, 0)}")


print(f"Class detected: {train_dataset.classes}")

print("Augmentation summary per class:")
for cls in train_dataset.classes:
    print(f"{cls.ljust(15)} → {'Strong' if cls in transform_dict else 'Default'}")

print(f"Total train batches: {len(train_dataloader)}")
print(f"Total validation batches: {len(val_dataloader)}")

Train sample counts per class:
algal_spot: 1465
brown-blight: 1397
gray-blight: 1220
healthy: 755
helopeltis: 1351
leaf-rust: 1600
red-rust: 417
red-spider-infested: 732
red-spot: 755
white-spot: 233
Validation sample counts per class:
algal_spot: 100
brown-blight: 60
gray-blight: 105
healthy: 45
helopeltis: 155
leaf-rust: 67
red-rust: 5
red-spider-infested: 14
red-spot: 45
white-spot: 32
Class detected: ['algal_spot', 'brown-blight', 'gray-blight', 'healthy', 'helopeltis', 'leaf-rust', 'red-rust', 'red-spider-infested', 'red-spot', 'white-spot']
Augmentation summary per class:
algal_spot      → Default
brown-blight    → Default
gray-blight     → Default
healthy         → Default
helopeltis      → Default
leaf-rust       → Default
red-rust        → Default
red-spider-infested → Default
red-spot        → Strong
white-spot      → Default
Total train batches: 1241
Total validation batches: 79


In [24]:
# Load YOLO11 classification model with pre-trained weights
model = YOLO("yolo11n.pt")  # Nano classification model, pre-trained on ImageNet
# Note: YOLO11 handles classifier adjustment internally during training, no manual layer modification needed

In [25]:
num_classes = len(train_dataset.classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('is cuda available?', torch.cuda.is_available())
model = model.to(device)

is cuda available? True


In [26]:
class_weights = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 1.0, 1.0]).to(device)  # Lower weight for red-spider-infested
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
num_epochs = 10
training_args = {
    'data': '../../dataset/Train/',  # Correct dataset path
    'epochs': num_epochs,
    'val': '../../dataset/Valid/',
    'nc': 10,
    'names': ['algal_spot', 'brown-blight', 'gray-blight', 'healthy', 'helopeltis', 'leaf-rust', 'red-rust', 'red-spider-infested', 'red-spot', 'white-spot'],
    'imgsz': 224,  # std for classification
    'batch': 8,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 0,  # Match your setup
    'project': './runs/train',
    'name': 'yolo11_cls_exp_new',
    'exist_ok': True,
    'pretrained': True,
    'optimizer': 'Adam',
    'lr0': 0.0001,
    'patience': 50,
    # Augmentation settings (respecting red-spot's strong augmentation)
    'hsv_h': 0.015,  # Default hue
    'hsv_s': 0.7,    # Default saturation
    'hsv_v': 0.4,    # Default value
    'degrees': 10.0,  # Rotation
    'translate': 0.1, # Translation
    'scale': 0.5,    # Zoom
    'shear': 0.0,
    'flipud': 0.0,   # Vertical flip
    'fliplr': 0.5,   # Horizontal flip
    'mosaic': 0.0,   # Disable mosaic for classification
    'mixup': 0.0,    # Disable mixup
    'cls_weight': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 1.0, 1.0]  # Reduce weight for red-spider-infested
}

Starting epoch 1/10
New https://pypi.org/project/ultralytics/8.3.115 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

RuntimeError: Dataset '/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml' error ❌ '/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml' does not exist